In [1]:
%load_ext lab_black

In [1]:
%%writefile modeling.py

import numpy as np
import tensorflow as tf
from tensorflow.keras import activations, initializers, regularizers, Model
from tensorflow.keras.layers import Layer, Input
import tensorflow.keras.backend as K

# Older Semantic
# def input_s(e, t, f, i,
#             gf=4, gi=1,
#             kf=100, ki=100,
#             hf=5, hi=5,
#             tmax=3.8,
#             mf=4.4743, sf=2.4578,
#             mi=4.1988, si=1.0078):

#     zf = (np.log(f + 2) - mf) / sf
#     numer_f = gf * e * (zf + hf)
#     denom_f = e * (zf + hf) + kf

#     zi = (i - mi) / si
#     numer_i = gi * e * (zi + hi)
#     denom_i = e * (zi + hi) + ki

#     return (t / tmax) * ((numer_f / denom_f) + (numer_i / denom_i))

def input_s(e, t, f, i, gf, gi, kf, ki, tmax=3.8,
            mf=4.4743, sf=2.4578, mi=4.1988, si=1.0078, hf=0, hi=0):
    # Semantic refresh V1

    numer_f = gf * e * np.log(f+2)
    denom_f = e * np.log(f+2) + kf

    return (t/tmax)*(numer_f/denom_f)

# def input_s(e,
#             t,
#             f,
#             i,
#             gf,
#             gi,
#             kf,
#             ki,
#             tmax=3.8,
#             mf=4.4743,
#             sf=2.4578,
#             mi=4.1988,
#             si=1.0078,
#             hf=5,
#             hi=5):
#     # Semantic refresh V2
#     numer_f = gf * np.sqrt(e) * np.log(f + 2)
#     denom_f = np.sqrt(e) * np.log(f + 2) + kf

#     return (t / tmax) * (numer_f / denom_f)


# def input_s(
#     e,
#     t,
#     f,
#     i,
#     gf,
#     gi,
#     kf,
#     ki,
#     tmax=3.8,
#     mf=4.4743,
#     sf=2.4578,
#     mi=4.1988,
#     si=1.0078,
#     hf=5,
#     hi=5
# ):
#     # Semantic refresh V3
#     numer_e = gf * e
#     denom_e = e + kf

#     return (t / tmax) * ((numer_e / denom_e) + 0.1 * np.log(f + 2))


class rnn(Layer):
    # Use keras rnn layer seems more efficient, maybe upgrade later...
    def __init__(self, cfg, input_p_dignostic=False, name='rnn', **kwargs):

        super(rnn, self).__init__(**kwargs)

        self.cfg = cfg
        self.input_p_dignostic = input_p_dignostic

        self.rnn_activation = activations.get(self.cfg.rnn_activation)
        self.weight_regularizer = regularizers.l2(cfg.regularizer_const)

        self.w_oh = self.add_weight(
            name='w_oh',
            shape=(self.cfg.o_input_dim, self.cfg.hidden_units),
            initializer=self.cfg.w_initializer,
            regularizer=self.weight_regularizer,
            trainable=True
        )

        self.w_hp = self.add_weight(
            name='w_hp',
            shape=(self.cfg.hidden_units, self.cfg.pho_units),
            initializer=self.cfg.w_initializer,
            regularizer=self.weight_regularizer,
            trainable=True
        )

        self.w_pp = self.add_weight(
            name='w_pp',
            shape=(self.cfg.pho_units, self.cfg.pho_units),
            initializer=self.cfg.w_initializer,
            regularizer=self.weight_regularizer,
            trainable=True
        )

        self.w_pc = self.add_weight(
            name='w_pc',
            shape=(self.cfg.pho_units, self.cfg.cleanup_units),
            initializer=self.cfg.w_initializer,
            regularizer=self.weight_regularizer,
            trainable=True
        )

        self.w_cp = self.add_weight(
            name='w_cp',
            shape=(self.cfg.cleanup_units, self.cfg.pho_units),
            initializer=self.cfg.w_initializer,
            regularizer=self.weight_regularizer,
            trainable=True
        )

        self.bias_h = self.add_weight(
            shape=(self.cfg.hidden_units, ),
            name='bias_h',
            initializer='zeros',
            regularizer=self.weight_regularizer,
            trainable=True
        )

        self.bias_p = self.add_weight(
            shape=(self.cfg.pho_units, ),
            name='bias_p',
            initializer='zeros',
            regularizer=self.weight_regularizer,
            trainable=True
        )

        self.bias_c = self.add_weight(
            shape=(self.cfg.cleanup_units, ),
            name='bias_c',
            initializer='zeros',
            regularizer=self.weight_regularizer,
            trainable=True
        )

    def call(self, inputs):
        # If input_p_dignostic = True, it will output input_p instead of act_p (for troubleshooting)
        # Hack for complying keras.layers.concatenate() format
        # Dimension note: (batch, timestep, input_dim)
        # Spliting input_dim below (index = 2)
        if self.cfg.use_semantic == True:
            o_input, s_input = tf.split(
                inputs, [self.cfg.o_input_dim, self.cfg.pho_units], 2
            )
        else:
            o_input = inputs

        ### Trial level init ###
        self.input_h_list = []
        self.input_p_list = []
        self.input_c_list = []

        self.act_h_list = []
        self.act_p_list = []
        self.act_c_list = []

        # Set inputs to 0
        self.input_h_list.append(
            tf.zeros((1, self.cfg.hidden_units), dtype=tf.float32)
        )
        self.input_p_list.append(
            tf.zeros((1, self.cfg.pho_units), dtype=tf.float32)
        )
        self.input_c_list.append(
            tf.zeros((1, self.cfg.cleanup_units), dtype=tf.float32)
        )

        # Set activations to 0.5
        self.act_h_list.append(self.input_h_list[0] + 0.5)
        self.act_p_list.append(self.input_p_list[0] + 0.5)
        self.act_c_list.append(self.input_c_list[0] + 0.5)

        for t in range(1, self.cfg.n_timesteps + 1):
            # Inject noise to weights in each time step
            # Method 1: Inject noise at each time step with reset
            if self.cfg.w_oh_noise != 0:
                w_oh = self.inject_noise(self.w_oh, self.cfg.w_oh_noise)
            else:
                w_oh = self.w_oh

            if self.cfg.w_hp_noise != 0:
                w_hp = self.inject_noise(self.w_hp, self.cfg.w_hp_noise)
            else:
                w_hp = self.w_hp

            if self.cfg.w_pp_noise != 0:
                w_pp = self.inject_noise(self.w_pp, self.cfg.w_pp_noise)
            else:
                w_pp = self.w_pp

            if self.cfg.w_pc_noise != 0:
                w_pc = self.inject_noise(self.w_pc, self.cfg.w_pc_noise)
            else:
                w_pc = self.w_pc

            if self.cfg.w_cp_noise != 0:
                w_cp = self.inject_noise(self.w_cp, self.cfg.w_cp_noise)
            else:
                w_cp = self.w_cp

            ##### Hidden layer #####
            oh = tf.matmul(o_input[:, t - 1, :], w_oh)
            mem_h = self.input_h_list[t - 1]
            h = self.cfg.tau * (oh + self.bias_h) + (1 - self.cfg.tau) * mem_h

            self.input_h_list.append(h)
            self.act_h_list.append(self.rnn_activation(h))

            ##### Phonology layer #####
            hp = tf.matmul(self.act_h_list[t - 1], w_hp)
            pp = tf.matmul(
                self.act_p_list[t - 1],
                tf.linalg.set_diag(w_pp, tf.zeros(self.cfg.pho_units))
            )  # Zero diagonal lock
            cp = tf.matmul(self.act_c_list[t - 1], w_cp)

            mem_p = self.input_p_list[t - 1]

            if self.cfg.use_semantic == True:  # Inject semantic input
                sp = s_input[:, t - 1, :]
            else:
                sp = 0

            p = self.cfg.tau * (hp + pp + cp + sp +
                                self.bias_p) + (1 - self.cfg.tau) * mem_p

            self.input_p_list.append(p)
            self.act_p_list.append(self.rnn_activation(p))

            ##### Cleanup layer #####
            pc = tf.matmul(self.act_p_list[t - 1], w_pc)
            mem_c = self.input_c_list[t - 1]
            c = self.cfg.tau * (pc + self.bias_c) + (1 - self.cfg.tau) * mem_c

            self.input_c_list.append(c)
            self.act_c_list.append(self.rnn_activation(c))

        if self.input_p_dignostic == True:
            return self.input_p_list[1:]
        else:
            return self.act_p_list[1:]

    def inject_noise(self, x, noise_sd):
        noise = K.random_normal(shape=K.shape(x), mean=0., stddev=noise_sd)
        return x + noise

    def compute_output_shape(self):
        return tensor_shape.as_shape(
            [1, self.cfg.pho_units] + self.cfg.n_timesteps
        )

    def get_config(self):
        config = {'custom_cfg': self.cfg}
        base_config = super(rnn, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


class attractor_rnn(Layer):
    def __init__(self, cfg, clamp_steps=14, **kwargs):
        super(attractor_rnn, self).__init__(**kwargs)
        self._name = 'rnn'
        self.cfg = cfg
        self.clamp_steps = clamp_steps
        self.rnn_activation = activations.get(self.cfg.rnn_activation)

    def build(self, input_shape, **kwargs):

        self.w_pp = self.add_weight(
            name='w_pp',
            shape=(self.cfg.pho_units, self.cfg.pho_units),
            initializer=self.cfg.w_initializer,
            trainable=True
        )

        self.w_pc = self.add_weight(
            name='w_pc',
            shape=(self.cfg.pho_units, self.cfg.cleanup_units),
            initializer=self.cfg.w_initializer,
            trainable=True
        )

        self.w_cp = self.add_weight(
            name='w_cp',
            shape=(self.cfg.cleanup_units, self.cfg.pho_units),
            initializer=self.cfg.w_initializer,
            trainable=True
        )

        self.bias_p = self.add_weight(
            shape=(self.cfg.pho_units, ),
            name='bias_p',
            initializer='zeros',
            trainable=True
        )

        self.bias_c = self.add_weight(
            shape=(self.cfg.cleanup_units, ),
            name='bias_c',
            initializer='zeros',
            trainable=True
        )

    def call(self, inputs):

        import tensorflow as tf

        ### Trial level init ###
        self.input_p_list = []
        self.input_c_list = []

        self.act_p_list = []
        self.act_c_list = []

        # Initialize input at step 0 to 3*input
        self.input_p_list.append(inputs * 3)
        self.input_c_list.append(
            tf.zeros((1, self.cfg.cleanup_units), dtype=tf.float32)
        )

        # Initialize activations
        self.act_p_list.append(self.rnn_activation(self.input_p_list[0]))
        self.act_c_list.append(self.input_c_list[0] + 0.5)

        for t in range(1, self.cfg.n_timesteps + 1):

            # ##### Phonology layer #####
            pp = tf.matmul(
                self.act_p_list[t - 1],
                tf.linalg.set_diag(self.w_pp, tf.zeros(self.cfg.pho_units))
            )  # Zero diagonal lock
            cp = tf.matmul(self.act_c_list[t - 1], self.w_cp)

            mem_p = self.input_p_list[t - 1]
            p = self.cfg.tau * (pp + cp +
                                self.bias_p) + (1 - self.cfg.tau) * mem_p

            self.input_p_list.append(p)

            if self.cfg.n_timesteps <= self.clamp_steps:
                act_p = inputs
            else:
                act_p = self.rnn_activation(p)

            self.act_p_list.append(act_p)

            ##### Cleanup layer #####
            pc = tf.matmul(self.act_p_list[t - 1], self.w_pc)

            mem_c = self.input_c_list[t - 1]
            c = self.cfg.tau * (pc + self.bias_c) + (1 - self.cfg.tau) * mem_c

            self.input_c_list.append(c)
            self.act_c_list.append(self.rnn_activation(c))

        return self.act_p_list[self.clamp_steps + 1:
                              ]  # Can get forgetting curve?

    def compute_output_shape(self):
        n = self.cfg.n_timesteps - self.clamp_steps
        return tensor_shape.as_shape([1, self.cfg.pho_units] + n)

    def get_config(self):
        config = {'custom_cfg': self.cfg}
        base_config = super(attractor_rnn, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


class attractor():
    # Attractor class is a model level object
    # Since we had used a non-serializable custom rnn layer... we need to rebuild the model using build_model()
    # If the attractor structure change please update build_model()

    def __init__(self, attractor_cfg, h5_name):

        self.cfg = attractor_cfg
        self.build_model()
        self.model.summary()

        self.model.load_weights(self.cfg.path_weight_folder + h5_name)
        rnn_layer = self.model.get_layer('rnn')
        names = [weight.name for weight in rnn_layer.weights]
        weights = self.model.get_weights()

        for name, weight in zip(names, weights):
            if name.endswith('w_pp:0'):
                self.pretrained_w_pp = weight
            if name.endswith('w_pc:0'):
                self.pretrained_w_pc = weight
            if name.endswith('w_cp:0'):
                self.pretrained_w_cp = weight
            if name.endswith('bias_p:0'):
                self.pretrained_bias_p = weight
            if name.endswith('bias_c:0'):
                self.pretrained_bias_c = weight

    def build_model(self):
        clamp_steps = 14
        input_o = Input(shape=(self.cfg.pho_units, ))
        rnn_model = attractor_rnn(self.cfg, clamp_steps)(input_o)
        self.model = Model(input_o, rnn_model)


def arm_attractor(model, attractor):
    # This function will load attractor weights (w_pp, w_pc, w_cp, bias_p, and bias_c) to model

    n_matrices = len(model.get_layer('rnn').weights)
    new_weights = []

    for i in range(n_matrices):
        # Align model and attractor weight matrices by creating new_weights list

        # Get attractor value if weight matrix name match attractor
        if model.get_layer('rnn').weights[i].name.endswith('w_pp:0'):
            new_weights.append(attractor.pretrained_w_pp)

        if model.get_layer('rnn').weights[i].name.endswith('w_pc:0'):
            new_weights.append(attractor.pretrained_w_pc)

        if model.get_layer('rnn').weights[i].name.endswith('w_cp:0'):
            new_weights.append(attractor.pretrained_w_cp)

        if model.get_layer('rnn').weights[i].name.endswith('bias_p:0'):
            new_weights.append(attractor.pretrained_bias_p)

        if model.get_layer('rnn').weights[i].name.endswith('bias_c:0'):
            new_weights.append(attractor.pretrained_bias_c)

        # Fill original value if this slot have not been filled
        if len(new_weights) < i + 1:
            new_weights.append(model.get_weights()[i])

    model.set_weights(new_weights)
    return model

Overwriting modeling.py
